<a href="https://colab.research.google.com/github/358Xin/DL/blob/main/MAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [216]:
!pip install qqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [217]:
workspace_dir = '.'
!gdown --id 1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U --output "{workspace_dir}/Omniglot.tar.gz"

!tar -zxf "{workspace_dir}/Omniglot.tar.gz" -C "{workspace_dir}/"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U
To: /content/Omniglot.tar.gz
100% 26.4M/26.4M [00:00<00:00, 71.0MB/s]


In [218]:
from PIL import Image
from IPython.display import display

for i in range(10, 20):
  im = Image.open("Omniglot/images_background/Japanese_(hiragana).0/character13/0500_" + str (i) + ".png")
  display(im)

In [219]:
import glob, random
from collections import OrderedDict
import numpy as np
from qqdm import qqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from IPython.display import display

device = "cuda" if torch.cuda.is_available() else "cpu"

random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(random_seed)

In [220]:
def ConvBlock(in_ch:int, out_ch:int):
  return nn.Sequential(
      nn.Conv2d(in_ch, out_ch, 3, padding=1),
      nn.BatchNorm2d(out_ch),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
  )

def ConvBlockFunction(x, w, b, w_bn, b_bn):
  x = F.conv2d(x, w, b, padding=1)
  x = F.batch_norm(x, running_mean=None, running_var=None, weight=w_bn, bias=b_bn, training=True)
  x = F.relu(x)
  x = F.max_pool2d(x, kernel_size=2, stride=2)
  return x

In [221]:
from torch.nn.modules.conv import Conv2d
class Classifier(nn.Module):
  def __init__(self, in_ch, n_way):
    super(Classifier, self).__init__()
    self.conv1 = ConvBlock(in_ch, 64)
    self.conv2 = ConvBlock(64, 64)
    self.conv3 = ConvBlock(64, 64)
    self.conv4 = ConvBlock(64, 64)
    self.logits = nn.Linear(64, n_way)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = x.view(x.shape[0], -1)
    x = self.logits(x)
    return x
  
  def functional_forward(self, x, params):
    for block in [1,2,3,4]:
      x = ConvBlockFunction(
          x, 
          params[f'conv{block}.0.weight'],
          params[f'conv{block}.0.bias'],
          params.get(f'conv{block}.1.weight'),
          params.get(f'conv{block}.1.bias'))
    x = x.view(x.shape[0], -1)
    x = F.linear(x, params['logits.weight'], params['logits.bias'])
    return x

In [222]:
def create_label(n_way, k_shot):
  return (torch.arange(n_way).repeat_interleave(k_shot).long())
create_label(5, 2)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

In [223]:
def calculate_accuracy(logits, val_label):
  acc = np.asarray([(torch.argmax(logits,-1).cpu().numpy() == val_label.cpu().numpy())]).mean()
  return acc

In [224]:
class Omniglot(Dataset):
  def __init__(self, data_dir, n_way, q_query):
    self.file_list = [f for f in glob.glob(data_dir + "**/character*", recursive=True)]
    self.transform = transforms.Compose([transforms.ToTensor()])
    self.m = n_way + q_query
  
  def __getitem__(self, index):
    sample = np.arange(20)
    np.random.shuffle(sample)
    img_path = self.file_list[index]
    img_list = [f for f in glob.glob(img_path + "**/*.png", recursive=True)]
    img_list.sort()
    imgs = [self.transform(Image.open(img_file)) for img_file in img_list]
    imgs = torch.stack(imgs)[sample[:self.m]]
    return imgs
  
  def __len__(self):
    return len(self.file_list)

In [225]:
def OriginalMAML(model, optimizer, x, n_way, k_shot, q_query, loss_fn, inner_train_step=1, inner_lr=0.4, train=True):
  criterion, task_loss, task_acc = loss_fn, [], []
  for meta_batch in x:
    support_set = meta_batch[:n_way*k_shot]
    query_set = meta_batch[n_way*k_shot:]

    fast_weights = OrderedDict(model.named_parameters())
    for inner_step in range(inner_train_step):
      train_label = create_label(n_way, k_shot).to(device)
      logits = model.functional_forward(support_set, fast_weights)
      loss = criterion(logits, train_label)
      grads = torch.autograd.grad(loss, fast_weights.values(), create_graph=True)
      fast_weights = OrderedDict((name, param-inner_lr*grad) for ((name, param), grad) in zip(fast_weights.items(), grads))
    
    val_label = create_label(n_way, k_shot).to(device)
    logits = model.functional_forward(query_set, fast_weights)
    loss = criterion(logits, val_label)
    task_loss.append(loss)
    task_acc.append(calculate_accuracy(logits, val_label))
  model.train()
  optimizer.zero_grad()

  meta_batch_loss = torch.stack(task_loss).mean()
  if train:
    meta_batch_loss.backward()
    optimizer.step()
  task_acc = np.mean(task_acc)
  return task_loss, task_acc

In [226]:
def inner_update_MAML(fast_weights, loss, inner_lr):
  grads = torch.autograd.grad(loss, fast_weights.values(), create_graph=True)
  fast_weights = OrderedDict((name, param-inner_lr*grad) for ((name, param), grad) in zip(fast_weights.items(), grads))
  return fast_weights

In [227]:
def inner_update_alt1(fast_weights, loss, inner_lr):
  grads = torch.autograd.grad(loss, fast_weights.values(), create_graph=True)
  fast_weights = OrderedDict((name, param-inner_lr*grad) for ((name, param), grad) in zip(fast_weights.items(), grads))
  return fast_weights

def inner_update_alt2(fast_weights, loss, inner_lr):
  grads = torch.autograd.grad(loss, list(fast_weights.values())[-2:], create_graph=True)
  for ((name, param), grad) in zip(list(fast_weights.items())[-2:], grads):
    fast_weights[name] = param - inner_lr * grad
  return fast_weights

In [228]:
def collect_gradients_MAML(special_grad:OrderedDict, fast_weights, model, len_data):
  return special_grad

In [229]:
def collect_gradients_alt(special_grad:OrderedDict, fast_weights, model, len_data):
  diff = OrderedDict((name, param-fast_weights[name]) for (name, param) in model.named_parameters())
  for name in diff:
    special_grad[name] = special_grad.get(name, 0) + diff[name] / len_data
  return special_grad

In [230]:
def outer_update_MAML(model, meta_batch_loss, grad_tensors):
  meta_batch_loss.backward()

In [231]:
def outer_update_alt(model, meta_batch_loss, grad_tensors):
  for (name, param) in model.named_parameters():
    param.grad = grad_tensors[name]

In [232]:
def MetaAlgorithmGenerator(
    inner_update = inner_update_MAML,
    collect_gradients = collect_gradients_MAML,
    outer_update = outer_update_MAML):
  
  global calculate_accuracy
  def MetaAlgorithm(model, optimizer, x, n_way, k_shot, q_query, loss_fn, inner_train_step=1, inner_lr=0.4, train=True):
    criterion, task_loss, task_acc = loss_fn, [], []
    special_grad = OrderedDict()

    for meta_batch in x:
      support_set = meta_batch[:n_way*k_shot]
      query_set = meta_batch[n_way*k_shot:]

      fast_weights = OrderedDict(model.named_parameters())
      
      for inner_step in range(inner_train_step): 
        train_label = create_label(n_way, k_shot).to(device)
        logits = model.functional_forward(support_set, fast_weights)
        loss = criterion(logits, train_label)
        fast_weights = inner_update(fast_weights, loss, inner_lr)
      
      val_label = create_label(n_way, q_query).to(device)
      special_grad = collect_gradients(special_grad, fast_weights, model, len(x))
      logits = model.functional_forward(query_set, fast_weights)
      loss = criterion(logits, val_label)
      task_loss.append(loss)
      task_acc.append(calculate_accuracy(logits, val_label))
    
    model.train()
    optimizer.zero_grad()

    meta_batch_loss = torch.stack(task_loss).mean()

    if train:
      outer_update(model, meta_batch_loss, special_grad)
      optimizer.step()
    task_acc = np.mean(task_acc)
    return meta_batch_loss, task_acc
  return MetaAlgorithm

In [233]:
Give_me_the_answer = True
def HiddenAnswer():
  MAML = MetaAlgorithmGenerator()
  FOMAML = MetaAlgorithmGenerator(inner_update=inner_update_alt1)
  ANIL = MetaAlgorithmGenerator(inner_update=inner_update_alt2)
  return MAML, FOMAML, ANIL

In [234]:
if Give_me_the_answer:
  MAML, FOMAML, ANIL = HiddenAnswer()
else:
  MAML = MetaAlgorithmGenerator()
  FOMAML = MetaAlgorithmGenerator()
  ANIL = MetaAlgorithmGenerator()

In [235]:
n_way = 5
k_shot = 1
q_query = 1
inner_train_step = 1
inner_lr = 0.4
meta_lr = 0.001
meta_batch_size = 32
max_epoch = 30
train_data_path = './Omniglot/images_background/'
test_data_path = './Omniglot/images_evaluation/'

In [236]:
def dataloader_init(datasets, num_workers=2):
  train_set, val_set, test_set = datasets
  train_loader = DataLoader(train_set, batch_size=n_way, num_workers=num_workers, shuffle=True, drop_last=True)
  val_loader = DataLoader(val_set, batch_size=n_way, num_workers=num_workers, shuffle=True, drop_last=True)
  test_loader = DataLoader(test_set, batch_size=n_way, num_workers=num_workers, shuffle=True, drop_last=True)

  train_iter = iter(train_loader)
  val_iter = iter(val_loader)
  test_iter = iter(test_loader)
  return (train_loader, val_loader, test_loader), (train_iter, val_iter, test_iter)

train_set, val_set = torch.utils.data.random_split(Omniglot(train_data_path, k_shot, q_query), [3200, 656])
test_set = Omniglot(test_data_path, k_shot, q_query)

(train_loader, val_loader, test_loader), (train_iter, val_iter, test_iter) = dataloader_init((train_set, val_set, test_set))

In [237]:
def model_init():
  meta_model = Classifier(1, n_way).to(device)
  optimizer = torch.optim.Adam(meta_model.parameters(), lr=meta_lr)
  loss_fn = nn.CrossEntropyLoss().to(device)
  return meta_model, optimizer, loss_fn

meta_model, optimizer, loss_fn = model_init()

In [238]:
def get_meta_batch(meta_batch_size, k_shot, q_query, data_loader, iterator):
  data = []
  for _ in range(meta_batch_size):
    try:
      task_data = iterator.next()
    except StopIteration:
      iterator = iter(data_loader)
      task_data = iterator.next()
    train_data = (task_data[:,:k_shot].reshape(-1, 1, 28, 28))
    val_data = (task_data[:,k_shot:].reshape(-1, 1, 28, 28))
    task_data = torch.cat((train_data, val_data), 0)
    data.append(task_data)
  return torch.stack(data).to(device), iterator

In [239]:
MetaAlgorithm = MAML

In [240]:
from tqdm import tqdm
for epoch in range(max_epoch):
  print("Epoch %d" % (epoch+1))
  train_meta_loss = []
  train_acc = []

  for step in tqdm(range(len(train_loader) // meta_batch_size)):
    x, train_iter = get_meta_batch(meta_batch_size, k_shot, q_query, train_loader, train_iter)
    meta_loss, acc = MetaAlgorithm(meta_model, optimizer, x, n_way, k_shot, q_query, loss_fn)
    train_meta_loss.append(meta_loss.item())
    train_acc.append(acc)

  print("Loss :", "%.3f" % (np.mean(train_meta_loss)), end='\t')
  print("Accuracy :", "%.3f %%" % (np.mean(train_acc) * 100))

  val_acc = []
  for step in tqdm(range(len(val_loader) // (meta_batch_size))):
    x, val_iter = get_meta_batch(meta_batch_size, k_shot, q_query, val_loader, val_iter)
    _, acc = MetaAlgorithm(meta_model, optimizer, x, n_way, k_shot, q_query, loss_fn, inner_train_step=3, train=False)
    val_acc.append(acc)
  print("Validation accuracy: ", "%.3f %%" % (np.mean(val_acc) * 100))

Epoch 1


100%|██████████| 20/20 [00:22<00:00,  1.15s/it]


Loss : 1.554	Accuracy : 45.188 %


100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


Validation accuracy:  46.406 %
Epoch 2


...

Validation accuracy:  96.094 %
Epoch 30


100%|██████████| 20/20 [00:18<00:00,  1.05it/s]


Loss : 0.141	Accuracy : 95.969 %


100%|██████████| 4/4 [00:03<00:00,  1.07it/s]

Validation accuracy:  96.250 %


In [241]:
test_acc = []
for step in tqdm(range(len(test_loader) // (meta_batch_size))):
  x, test_iter = get_meta_batch(meta_batch_size, k_shot, q_query, test_loader, test_iter)
  _, acc = MetaAlgorithm(meta_model, optimizer, x, n_way, k_shot, q_query, loss_fn, inner_train_step=3, train=False)
  test_acc.append(acc)  
print("Validation accuracy: ", "%.3f %%" % (np.mean(test_acc) * 100))

100%|██████████| 4/4 [00:03<00:00,  1.12it/s]

Validation accuracy:  92.656 %
